In [1]:
import sys 
import numpy as np
import pandas as pd
import sklearn
import sklearn
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, KFold
import math
import datetime
import os
import json

In [2]:
input_training = pd.read_csv("training.csv")
input_validation = pd.read_csv("validation.csv")

In [3]:
input_training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2100 entries, 0 to 2099
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   movie_id              2100 non-null   int64  
 1   cast                  2100 non-null   object 
 2   crew                  2100 non-null   object 
 3   budget                2100 non-null   int64  
 4   genres                2100 non-null   object 
 5   homepage              955 non-null    object 
 6   keywords              2100 non-null   object 
 7   original_language     2100 non-null   object 
 8   original_title        2100 non-null   object 
 9   overview              2100 non-null   object 
 10  production_companies  2100 non-null   object 
 11  production_countries  2100 non-null   object 
 12  release_date          2100 non-null   object 
 13  revenue               2100 non-null   int64  
 14  runtime               2100 non-null   float64
 15  spoken_languages     

In [4]:
input_training.head()

movie_id                                               cast  \
0     19995  [{"cast_id": 242, "character": "Jake Sully", "...   
1       285  [{"cast_id": 4, "character": "Captain Jack Spa...   
2    206647  [{"cast_id": 1, "character": "James Bond", "cr...   
3     49026  [{"cast_id": 2, "character": "Bruce Wayne / Ba...   
4     49529  [{"cast_id": 5, "character": "John Carter", "c...   

                                                crew     budget  \
0  [{"credit_id": "52fe48009251416c750aca23", "de...  237000000   
1  [{"credit_id": "52fe4232c3a36847f800b579", "de...  300000000   
2  [{"credit_id": "54805967c3a36829b5002c41", "de...  245000000   
3  [{"credit_id": "52fe4781c3a36847f81398c3", "de...  250000000   
4  [{"credit_id": "52fe479ac3a36847f813eaa3", "de...  260000000   

                                              genres  \
0  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                                       homepage  \
0                   http://www.avatarmovie.com/   
1  http://disney.go.com/disneypictures/pirates/   
2   http://www.sonypictures.com/movies/spectre/   
3            http://www.thedarkknightrises.com/   
4          http://movies.disney.com/john-carter   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   
2  [{"id": 470, "name": "spy"}, {"id": 818, "name...                en   
3  [{"id": 849, "name": "dc comics"}, {"id": 853,...                en   
4  [{"id": 818, "name": "based on novel"}, {"id":...                en   

                             original_title  \
0                                    Avatar   
1  Pirates of the Caribbean: At World's End   
2                                   Spectre   
3                     The Dark Knight Rises   
4                               John Carter   

                                            overview  \
0  In the 22nd century, a paraplegic Marine is di...   
1  Captain Barbossa, long believed to be dead, ha...   
2  A cryptic message from Bond’s past sends him o...   
3  Following the death of District Attorney Harve...   
4  John Carter is a war-weary, former military ca...   

                                production_companies  \
0  [{"name": "Ingenious Film Partners", "id": 289...   
1  [{"name": "Walt Disney Pictures", "id": 2}, {"...   
2  [{"name": "Columbia Pictures", "id": 5}, {"nam...   
3  [{"name": "Legendary Pictures", "id": 923}, {"...   
4        [{"name": "Walt Disney Pictures", "id": 2}]   

                                production_countries release_date     revenue  \
0  [{"iso_3166_1": "US", "name": "United States o...   2009-12-10  2787965087   
1  [{"iso_3166_1": "US", "name": "United States o...   2007-05-19   961000000   
2  [{"iso_3166_1": "GB", "name": "United Kingdom"...   2015-10-26   880674609   
3  [{"iso_3166_1": "US", "name": "United States o...   2012-07-16  1084939099   
4  [{"iso_3166_1": "US", "name": "United States o...   2012-03-07   284139100   

   runtime                                   spoken_languages    status  \
0    162.0  [{"iso_639_1": "en", "name": "English"}, {"iso...  Released   
1    169.0           [{"iso_639_1": "en", "name": "English"}]  Released   
2    148.0  [{"iso_639_1": "fr", "name": "Fran\u00e7ais"},...  Released   
3    165.0           [{"iso_639_1": "en", "name": "English"}]  Released   
4    132.0           [{"iso_639_1": "en", "name": "English"}]  Released   

                                          tagline  rating  
0                     Enter the World of Pandora.       3  
1  At the end of the world, the adventure begins.       3  
2                           A Plan No O

In [5]:
def get_json_names(x):
    json_data = json.loads(x)
    list_genres = []
    for token in json_data:
        list_genres.append(token['name'])
    return list_genres

def format_training_data(df_training, df_validation):
    training_dataset = pd.DataFrame(columns=['movie_id'])
    validation_dataset = pd.DataFrame(columns=['movie_id'])
    # movie_id:
    training_dataset['movie_id'] = df_training['movie_id']
    validation_dataset['movie_id'] = df_validation['movie_id']
    
    # revenue:
    training_dataset['revenue'] = df_training['revenue']
    validation_dataset['revenue'] = df_validation['revenue']
    

    # budget
    training_dataset['budget'] = df_training['budget']
    validation_dataset['budget'] = df_validation['budget']
    
    # rating
    
    # homepage:
    training_dataset['has_homepage'] = 0
    training_dataset.loc[df_training['homepage'].isnull() == False, 'has_homepage'] = 1 #1 here means it has home page
    
    validation_dataset['has_homepage'] = 0
    validation_dataset.loc[df_validation['homepage'].isnull() == False, 'has_homepage'] = 1 #1 here means it has home page
    
    # original_language:
    unique_languages = list(df_training["original_language"].apply(pd.Series).stack().unique())
    for language in unique_languages :
        training_dataset[language] = df_training['original_language'].apply(lambda x: 1 if x == language else 0)
    for language in unique_languages :
        validation_dataset[language] = df_validation['original_language'].apply(lambda x: 1 if x == language else 0)
    
    # Genres: # may improve by looking at how correlated to revenue
#     unique_genres = list(training_dataset["genres"].apply(pd.Series).stack().unique())

    # get top 10 mean-revenue genres
#     temp = df_training[['movie_id','revenue','genres']].copy()
#     temp['genres'] = input_training['genres'].apply(lambda x: get_json_names(x))
#     unique_genres = list(temp["genres"].apply(pd.Series).stack().unique())

#     dic={}
#     for a in unique_genres:
#         mask = temp['genres'].apply(lambda x: a in x)
#         dic[a] = temp[mask]['revenue'].mean()

#     t = pd.DataFrame.from_dict(dic, orient='index', columns=['mean_revenue']).reset_index().rename(columns={'index':'genre'})
#     genres_list = list(t.sort_values(by=["mean_revenue"],ascending=False)["genre"])[:10]
    
    
    # get  10 most appearing
    temp = input_training[['movie_id','revenue','genres']].copy()
    temp['genres'] = temp['genres'].apply(lambda x: get_json_names(x))
    genres_list = list(temp['genres'].apply(pd.Series).stack().value_counts().index)[:10]
    
    
#     genres_list=['Drama','Action','Comedy','Thriller','Adventure','Crime','Romance','Science Fiction','Family','Fantasy']
    training_dataset['genres'] = df_training['genres'].apply(lambda x: get_json_names(x))
    for genres in genres_list :
        training_dataset['genre_'+genres]=training_dataset['genres'].apply(lambda x: 1 if genres in x else 0)
    training_dataset = training_dataset.drop(['genres'], axis=1)

    validation_dataset['genres'] = df_validation['genres'].apply(lambda x: get_json_names(x))
    for genres in genres_list :
        validation_dataset['genre_'+genres]=validation_dataset['genres'].apply(lambda x: 1 if genres in x else 0)
    validation_dataset = validation_dataset.drop(['genres'], axis=1)

    # Production company
#     production_company_list = ['Warner Bros.', 'Universal Pictures', 'Paramount Pictures', 'Twentieth Century Fox Film Corporation', 'Columbia Pictures']
    # top 10 companies who produce most movies
    temp = input_training[['movie_id','revenue','production_companies']].copy()
    temp['production_companies'] = input_training['production_companies'].apply(lambda x: get_json_names(x))
    production_company_list = list(temp['production_companies'].apply(pd.Series).stack().value_counts().index)[:10]
    
    training_dataset['production_companies'] = df_training['production_companies'].apply(lambda x: get_json_names(x))
    for company in production_company_list :
        training_dataset['company_'+company]=training_dataset['production_companies'].apply(lambda x: 1 if company in x else 0)
    training_dataset = training_dataset.drop(['production_companies'], axis=1)
    
    validation_dataset['production_companies'] = df_validation['production_companies'].apply(lambda x: get_json_names(x))
    for company in production_company_list :
        validation_dataset['company_'+company]=validation_dataset['production_companies'].apply(lambda x: 1 if company in x else 0)
    validation_dataset = validation_dataset.drop(['production_companies'], axis=1)
    
    
    # crew 
    temp = df_training[['movie_id','revenue','crew']].copy()
    temp['crew'] = temp['crew'].apply(lambda x: get_json_names(x))
    crew_list = list(temp['crew'].apply(pd.Series).stack().value_counts().index)[:10]
    
    training_dataset['crew'] = df_training['crew'].apply(lambda x: get_json_names(x))
    for crew in crew_list :
        training_dataset['crew_'+crew]=training_dataset['crew'].apply(lambda x: 1 if crew in x else 0)
    training_dataset = training_dataset.drop(['crew'], axis=1)
    
    validation_dataset['crew'] = df_validation['crew'].apply(lambda x: get_json_names(x))
    for crew in crew_list :
        validation_dataset['crew_'+crew]=validation_dataset['crew'].apply(lambda x: 1 if crew in x else 0)
    validation_dataset = validation_dataset.drop(['crew'], axis=1)
    
    # runtime
#     training_dataset['runtime'] = df_training['runtime']
#     validation_dataset['runtime'] = df_validation['runtime']
    training_dataset['runtime_cat_min_60'] = df_training['runtime'].apply(lambda x: 1 if (x <=60) else 0)
    training_dataset['runtime_cat_61_80'] = df_training['runtime'].apply(lambda x: 1 if (x >60)&(x<=80) else 0)
    training_dataset['runtime_cat_81_100'] = df_training['runtime'].apply(lambda x: 1 if (x >80)&(x<=100) else 0)
    training_dataset['runtime_cat_101_120'] = df_training['runtime'].apply(lambda x: 1 if (x >100)&(x<=120) else 0)
    training_dataset['runtime_cat_121_140'] = df_training['runtime'].apply(lambda x: 1 if (x >120)&(x<=140) else 0)
    training_dataset['runtime_cat_141_170'] = df_training['runtime'].apply(lambda x: 1 if (x >140)&(x<=170) else 0)
    training_dataset['runtime_cat_171_max'] = df_training['runtime'].apply(lambda x: 1 if (x >=170) else 0)
    
    validation_dataset['runtime_cat_min_60'] = df_validation['runtime'].apply(lambda x: 1 if (x <=60) else 0)
    validation_dataset['runtime_cat_61_80'] = df_validation['runtime'].apply(lambda x: 1 if (x >60)&(x<=80) else 0)
    validation_dataset['runtime_cat_81_100'] = df_validation['runtime'].apply(lambda x: 1 if (x >80)&(x<=100) else 0)
    validation_dataset['runtime_cat_101_120'] = df_validation['runtime'].apply(lambda x: 1 if (x >100)&(x<=120) else 0)
    validation_dataset['runtime_cat_121_140'] = df_validation['runtime'].apply(lambda x: 1 if (x >120)&(x<=140) else 0)
    validation_dataset['runtime_cat_141_170'] = df_validation['runtime'].apply(lambda x: 1 if (x >140)&(x<=170) else 0)
    validation_dataset['runtime_cat_171_max'] = df_validation['runtime'].apply(lambda x: 1 if (x >=170) else 0)
    
    # release date
    training_dataset['release_date'] = pd.to_datetime(df_training['release_date'])
    validation_dataset['release_date'] = pd.to_datetime(df_validation['release_date'])
    
    date_parts = ["year", "weekday", "month"]
    for part in date_parts:
        part_col = 'release_date' + "_" + part #add prefix as  "release_date" before the columne
        training_dataset[part_col] = getattr(training_dataset['release_date'].dt, part).astype(int)
        validation_dataset[part_col] = getattr(validation_dataset['release_date'].dt, part).astype(int)
    training_dataset = training_dataset.drop(['release_date'], axis=1)
    validation_dataset = validation_dataset.drop(['release_date'], axis=1)
    
    # keyword
    # get 10 most appearing
    temp = input_training[['movie_id','revenue','keywords']].copy()
    temp['keywords'] = temp['keywords'].apply(lambda x: get_json_names(x))
    keywords_list = list(temp['keywords'].apply(pd.Series).stack().value_counts().index)[:10]
    
    training_dataset['keywords'] = df_training['keywords'].apply(lambda x: get_json_names(x))
    for keyword in keywords_list :
        training_dataset['keyword_'+keyword]=training_dataset['keywords'].apply(lambda x: 1 if keyword in x else 0)
    training_dataset = training_dataset.drop(['keywords'], axis=1)

    validation_dataset['keywords'] = df_validation['keywords'].apply(lambda x: get_json_names(x))
    for keyword in keywords_list :
        validation_dataset['keyword_'+keyword]=validation_dataset['keywords'].apply(lambda x: 1 if keyword in x else 0)
    validation_dataset = validation_dataset.drop(['keywords'], axis=1)
    
    # production_countries
    # top 10 countries who produce most movies
#     temp = input_training[['movie_id','revenue','production_countries']].copy()
#     temp['production_countries'] = input_training['production_countries'].apply(lambda x: get_json_names(x))
#     production_countries_list = list(temp['production_countries'].apply(pd.Series).stack().value_counts().index)[:10]
    
#     training_dataset['production_countries'] = df_training['production_countries'].apply(lambda x: get_json_names(x))
#     for country in production_countries_list :
#         training_dataset['country_'+country]=training_dataset['production_countries'].apply(lambda x: 1 if country in x else 0)
#     training_dataset = training_dataset.drop(['production_countries'], axis=1)
    
#     validation_dataset['production_countries'] = df_validation['production_countries'].apply(lambda x: get_json_names(x))
#     for country in production_countries_list :
#         validation_dataset['country_'+country]=validation_dataset['production_countries'].apply(lambda x: 1 if country in x else 0)
#     validation_dataset = validation_dataset.drop(['production_countries'], axis=1)
    
    
    # cast
#     temp = df_training[['movie_id','revenue','cast']].copy()
#     temp['cast'] = temp['cast'].apply(lambda x: get_json_names(x))
#     cast_list = list(temp['cast'].apply(pd.Series).stack().value_counts().index)[:10]
    
#     training_dataset['cast'] = df_training['cast'].apply(lambda x: get_json_names(x))
#     for cast in cast_list :
#         training_dataset['cast_'+cast]=training_dataset['cast'].apply(lambda x: 1 if cast in x else 0)
#     training_dataset = training_dataset.drop(['cast'], axis=1)
    
#     validation_dataset['cast'] = df_validation['cast'].apply(lambda x: get_json_names(x))
#     for cast in cast_list :
#         validation_dataset['cast_'+cast]=validation_dataset['cast'].apply(lambda x: 1 if cast in x else 0)
#     validation_dataset = validation_dataset.drop(['cast'], axis=1)
    
    
    # spoken_languages
#     temp = df_training[['movie_id','revenue','spoken_languages']].copy()
#     temp['spoken_languages'] = temp['spoken_languages'].apply(lambda x: get_json_names(x))
#     spoken_languages_list = list(temp['spoken_languages'].apply(pd.Series).stack().value_counts().index)[:10]
    
#     training_dataset['spoken_languages'] = df_training['spoken_languages'].apply(lambda x: get_json_names(x))
#     for lang in spoken_languages_list :
#         training_dataset['spoken_languages_'+lang]=training_dataset['spoken_languages'].apply(lambda x: 1 if lang in x else 0)
#     training_dataset = training_dataset.drop(['spoken_languages'], axis=1)
    
#     validation_dataset['spoken_languages'] = df_validation['spoken_languages'].apply(lambda x: get_json_names(x))
#     for lang in spoken_languages_list :
#         validation_dataset['spoken_languages_'+lang]=validation_dataset['spoken_languages'].apply(lambda x: 1 if lang in x else 0)
#     validation_dataset = validation_dataset.drop(['spoken_languages'], axis=1)
    
    
    
    return training_dataset, validation_dataset 


In [6]:
training_dataset, validation_dataset = format_training_data(input_training, input_validation)

In [7]:
training_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2100 entries, 0 to 2099
Data columns (total 66 columns):
 #   Column                                          Non-Null Count  Dtype
---  ------                                          --------------  -----
 0   movie_id                                        2100 non-null   int64
 1   revenue                                         2100 non-null   int64
 2   budget                                          2100 non-null   int64
 3   has_homepage                                    2100 non-null   int64
 4   en                                              2100 non-null   int64
 5   ja                                              2100 non-null   int64
 6   fr                                              2100 non-null   int64
 7   zh                                              2100 non-null   int64
 8   ko                                              2100 non-null   int64
 9   te                                              2100 non-null  

In [8]:
# validation_dataset[validation_dataset['crew_Robert Rodriguez']==1]
validation_dataset

movie_id    revenue    budget  has_homepage  en  ja  fr  zh  ko  te  ...  \
0       19994   31556061  16000000             0   1   0   0   0   0   0  ...   
1       25166   15427192  16000000             0   1   0   0   0   0   0  ...   
2       12120   13854000  20000000             0   1   0   0   0   0   0  ...   
3        9800  206678440  26000000             0   1   0   0   0   0   0  ...   
4        1090   18564088  16000000             0   1   0   0   0   0   0  ...   
..        ...        ...       ...           ...  ..  ..  ..  ..  ..  ..  ...   
395      7942   33000000  10000000             1   1   0   0   0   0   0  ...   
396     73247    2154696  10000000             0   1   0   0   0   0   0  ...   
397    253331   21571189   9000000             0   1   0   0   0   0   0  ...   
398      9952    7177143  10000000             0   1   0   0   0   0   0  ...   
399    256924   10835752  10000000             0   1   0   0   0   0   0  ...   

     keyword_duringcreditsstinger  keyword_based on novel  \
0                               1                       0   
1                               0                       0   
2                               0                       0   
3                               0                       0   
4                               0                       0   
..                            ...                     ...   
395                             0                       0   
396                             0                       0   
397                             0                       0   
398                             0                       0   
399                             0                       0   

     keyword_aftercreditsstinger  keyword_dystopia  keyword_3d  \
0                              0                 0           0   
1                              0                 0           0   
2                              0                 0           0   
3                              0                 0           0   
4                              0                 1           0   
..                           ...               ...         ...   
395                            0                 0           0   
396                            0                 0           0   
397                            0                 0           0   
398                            0                 0           0   
399                            0                 0           0   

     keyword_violence  keyword_woman director  keyword_murder  keyword_sequel  \
0                   0                       1               0               0   
1                   0                       0               1               0   
2                   0                       0               0               0   
3                   0                       0               0               0   
4                   0                       0               1               0   
..                ...                     ...             ...             ...   
395                 0                       0               0               0   
396                 0                       0               0               0   
397                 0                       0               0               0   
398                 1                       0               0               0   
399                 0                       0               0               0   

     keyword_revenge  
0                  0  
1                  0  
2                  0  
3                  0  
4                  0  
..               ...  
395                0  
396                0  
397                0  
398                0  
399                0  

[400 rows x 66 columns]

In [22]:
train_X = training_dataset.drop(['revenue', 'movie_id'], axis=1)

# log
train_X['budget'] = train_X['budget'].apply(lambda x: math.log(x))
train_y = training_dataset['revenue'].apply(lambda x: math.log(x))

# #normal d
# train_y = training_dataset['revenue']

test_X = validation_dataset.drop(['revenue', 'movie_id'], axis=1)

# log
test_X['budget'] = test_X['budget'].apply(lambda x: math.log(x))
test_y = validation_dataset['revenue'].apply(lambda x: math.log(x))

# #normal
# train_y = training_dataset['revenue']

# X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, random_state = 1)
#linear regression
lm = LinearRegression() #our 6th model
lm.fit(train_X, train_y)
lm_preds = lm.predict(test_X)
a = mean_squared_error(lm_preds, test_y)
print ("R-Squared is:", r2_score(lm_preds, test_y))

R-Squared is: -4.247901794902349


In [23]:
from scipy.stats import pearsonr
corr, _ = pearsonr(np.exp(lm_preds), np.exp(test_y))
print('non_log Pearsons correlation: %.3f' % corr)
corr, _ = pearsonr(lm_preds, test_y)
print('log Pearsons correlation: %.3f' % corr)

non_log Pearsons correlation: 0.303
log Pearsons correlation: 0.410


In [24]:
import sklearn.metrics as metrics
from sklearn.ensemble import RandomForestRegressor

RF_model = RandomForestRegressor(random_state =0, n_estimators=500, max_depth=10)
RF_model.fit(train_X, train_y)

y_hat = RF_model.predict(test_X)
corr, _ = pearsonr(np.exp(y_hat), np.exp(test_y))
print('non_log Pearsons correlation: %.3f' % corr)
corr, _ = pearsonr(y_hat, test_y)
print('log Pearsons correlation: %.3f' % corr)

non_log Pearsons correlation: 0.294
log Pearsons correlation: 0.313


In [25]:
from sklearn import ensemble
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': .01, 'loss': 'ls'} 
clf = ensemble.GradientBoostingRegressor(**params)
predictions2 = clf.fit(train_X,train_y)
training_score = clf.score(train_X, train_y)
print(f"Training Score: {training_score}")

predictions2 = clf.predict(test_X).reshape(-1,1)
MSE = mean_squared_error(test_y, predictions2)
# r2 = clf.score(predictions2, test_y)
# print(f"MSE: {MSE}, R2: {r2}")
# corr, _ = pearsonr(np.exp(predictions2), np.exp(test_y))
# print('non_log Pearsons correlation: %.3f' % corr)
# corr, _ = pearsonr(predictions2, test_y)
# print('log Pearsons correlation: %.3f' % corr)
i = 0
preds=[]
while(i < len(predictions2)):
    preds.append(predictions2[i][0])
    i += 1
    
corr, _ = pearsonr(np.exp(preds), np.exp(test_y))
print('non_log Pearsons correlation: %.3f' % corr)
corr, _ = pearsonr(preds, test_y)
print('log Pearsons correlation: %.3f' % corr)

Training Score: 0.5851530371161149
non_log Pearsons correlation: 0.363
log Pearsons correlation: 0.362


In [41]:
res = pd.DataFrame(columns=['movie_id', 'preds', 'groundTruth'])

res['movie_id'] = validation_dataset['movie_id']
res['groundTruth'] = np.exp(test_y)
i = 0
while(i < len(preds)):
    res.loc[i, 'preds'] = np.exp(preds[i])
    i += 1


In [60]:
res

movie_id            preds  groundTruth  x_minus_xmean  y_minus_ymean  \
0       19994  54798109.415033   31556061.0   2.000703e+07  -2.655978e+07   
1       25166  34961415.299456   15427192.0   1.703326e+05  -4.268865e+07   
2       12120  35853501.620405   13854000.0   1.062419e+06  -4.426185e+07   
3        9800  61185798.947755  206678440.0   2.639472e+07   1.485626e+08   
4        1090  33748562.798403   18564088.0  -1.042520e+06  -3.955176e+07   
..        ...              ...          ...            ...            ...   
395      7942  35233715.864053   33000000.0   4.426331e+05  -2.511585e+07   
396     73247  15454530.237443    2154696.0  -1.933655e+07  -5.596115e+07   
397    253331   1128327.624055   21571189.0  -3.366276e+07  -3.654466e+07   
398      9952  25047131.351639    7177143.0  -9.743951e+06  -5.093870e+07   
399    256924  20714468.360473   10835752.0  -1.407661e+07  -4.728009e+07   

        x_times_y  
0   -5.313823e+14  
1   -7.271268e+12  
2   -4.702462e+13  
3    3.921268e+15  
4    4.123350e+13  
..            ...  
395 -1.111711e+13  
396  1.082096e+15  
397  1.230194e+15  
398  4.963442e+14  
399  6.655436e+14  

[400 rows x 6 columns]

In [53]:
n = len(res)
xmean = np.mean(res['preds'])
ymean = np.mean(res['groundTruth'])
res['x_minus_xmean'] = res['preds'].apply(lambda x: x-xmean)
res['y_minus_ymean'] = res['groundTruth'].apply(lambda x: x-ymean)

In [61]:
upper = sum(res['x_minus_xmean'] * res['y_minus_ymean'])
lower = math.sqrt(sum(res['x_minus_xmean']*res['x_minus_xmean'])*sum(res['y_minus_ymean']*res['y_minus_ymean']))
r = upper / lower
MSE = mean_squared_error(res['preds'], res['groundTruth'])

In [68]:
summary = pd.DataFrame(columns=['zid', 'MSE', 'correlation'], data=[['z5092195', round(MSE, 2), round(r, 2)]])

In [69]:
summary

zid           MSE  correlation
0  z5092195  7.900644e+15         0.36

In [129]:
from collections import Counter
temp = input_training[['movie_id','revenue','crew']].copy()
temp['crew'] = input_training['crew'].apply(lambda x: get_json_names(x))
crew_list = list(temp['crew'].apply(pd.Series).stack().value_counts().index)[:10]
# temp['crew'] = temp['crew'].apply(lambda x: x[:10])

# names = temp['crew'].sum()
# ctr = Counter(names)
# df_names = pd.DataFrame.from_dict(ctr, orient='index').reset_index().rename(columns={'index':'actor', 0:'count'})       
# df_names=df_names.sort_values('count', ascending=False)
# df_names = df_names[df_names['count'] > 9]

In [128]:
temp['crew'].apply(pd.Series).stack().value_counts()

Robert Rodriguez       83
Steven Spielberg       74
Hans Zimmer            63
James Newton Howard    55
Francine Maisler       55
                       ..
William Bonn            1
Mariano Carranco        1
Carlos Barbosa          1
Anupam Das              1
Giulio Tamassy          1
Length: 35208, dtype: int64

In [130]:
crew_list

['Robert Rodriguez',
 'Steven Spielberg',
 'Hans Zimmer',
 'James Newton Howard',
 'Francine Maisler',
 'Deborah Aquila',
 'Mary Vernieu',
 'Kevin Kaska',
 "Dan O'Connell",
 'Avy Kaufman']

In [245]:
temp = input_training[['movie_id','revenue','keywords']].copy()
temp['keywords'] = temp['keywords'].apply(lambda x: get_json_names(x))
keywords_list = list(temp['keywords'].apply(pd.Series).stack().value_counts().index)[:10]


In [246]:
temp['keywords'].apply(pd.Series).stack().value_counts()

duringcreditsstinger    210
based on novel          132
aftercreditsstinger     114
dystopia                100
3d                       87
                       ... 
wealth differences        1
stage                     1
people change             1
baader-meinhof group      1
insider                   1
Length: 6681, dtype: int64